## PnP算法
很多时候，当我们利用单目相机获取图像时，往往只能获取图像中特征物体的像素坐标。而在空间三维点计算时，我们也第一时间会想到深度相机（Kinect、Realsense等）、双目相机等。但是在实际中，我们亦可以通过单目相机来求得空间三维点，但这个往往有一个条件，已知特征物体的三个以上特征点参数

使用PnP（Perspective-n-Point）算法求解空间位姿。它解决了当我们知道n个3D空间点以及它们的投影位置时，如何估计相机所在的位姿的问题

## 实现方法

我们只要获得特征点的世界坐标（三维坐标）、2D坐标（像素坐标）、相机内参矩阵、相机畸变参数矩阵以上四个参数即可以解得相机与标志物之间的外参（R、T），并以此求得相机的世界坐标（以标志物为世界坐标平面，且原点为标志物已知某一点）

### 世界坐标

使用Aruco标记作为标志物，检测四个角点，通过PnP求解相机的位姿及空间坐标（以右手坐标系为基准，Z轴垂直于二维码平面往里）

使用Aruco标记的好处：

- 黑白对比度高，在各种光照条件下都能稳定检测
- opencv有封装好的函数可供检测Aruco标记
- 内置纠错机制，可以容忍部分遮挡
- 每个标记有唯一ID，避免混淆。支持同时检测多个标记

现在以Aruco标记的中心作为坐标原点，四个角点的世界坐标为（假设Aruco标记边长为marker_size）：

In [ ]:
import numpy as np
import cv2

def get_aruco_world_coordinates(marker_size=0.096):
    """
    返回ArUco标记四个角点的世界坐标(以标记中心为原点)
    Args:
        marker_size: 标记边长
    Returns:
        world_points: 世界坐标系下的四个角点坐标，形状为(4,3)的numpy数组
                      顺序为：左上、右上、右下、左下
    """
    half_size = marker_size / 2
    # 世界坐标（右手系，Z轴垂直标记平面向外）
    world_points = np.array([
        [-half_size, half_size, 0],   # 左上
        [half_size, half_size, 0],    # 右上  
        [half_size, -half_size, 0],   # 右下
        [-half_size, -half_size, 0]   # 左下
    ], dtype=np.float32)
    return world_points

### 图像坐标

使用opencv自带的Aruco检测函数检测图像中的Aruco标记，并获取其四个角点的像素坐标，这里我们采用4x4_50字典中的Aruco标记

函数说明详情参阅`Aruco标记检测.ipynb`

In [ ]:
def detect_aruco_corners(frame, aruco_dict=cv2.aruco.DICT_4X4_50):
    """
    检测图像中的ArUco标记四个角点
    Args:
        frame: 输入图像帧
        aruco_dict: ArUco字典类型, 默认为4x4_50
    Returns:
        corners: 四个角点的像素坐标，形状为(1,4,2)的numpy数组
                 顺序为：左上、右上、右下、左下
                 如果未检测到则返回None
    """
    # 获取ArUco字典
    aruco_dict = cv2.aruco.getPredefinedDictionary(aruco_dict)
    # 创建ArUco检测参数
    parameters = cv2.aruco.DetectorParameters()
    # 创建检测器
    detector = cv2.aruco.ArucoDetector(aruco_dict, parameters)
    # 检测ArUco标记
    corners, ids, _ = detector.detectMarkers(frame)
    # 如果检测到标记，返回第一个标记的四个角点
    if len(corners) > 0:
        return corners[0]
    return None

### 使用PnP求解位姿

使用检测到的Aruco标记的四个角点的像素坐标作为2D图像点，结合预定义的世界坐标点，通过 `cv2.solvePnP()` 求解相机的旋转向量和平移向量，从而得到相机在世界坐标系中的位置和姿态

函数原型：

`cv2.solvePnP(
    objectPoints, 
    imagePoints, 
    cameraMatrix, 
    distCoeffs, 
    rvec=None, 
    tvec=None, 
    useExtrinsicGuess=False, 
    flags=cv2.SOLVEPNP_ITERATIVE
)`

参数说明：

- 必需参数：
  - objectPoints: 世界坐标系中的n个3D点的数组。类型: np.ndarray, 形状为 (N, 3) 或 (N, 1, 3)。通常使用浮点类型 np.float32
  - imagePoints: 图像坐标系中与3D点对应的2D图像点坐标数组。类型: np.ndarray, 形状为 (N, 2) 或 (N, 1, 2)
  - cameraMatrix: 相机内参矩阵。类型: np.ndarray, 形状为 (3, 3)
  - distCoeffs: 畸变系数
- 可选参数：
  - rvec: 初始旋转向量（输入/输出）。当 useExtrinsicGuess=True 时作为初始值
  - tvec: 初始平移向量（输入/输出）。当 useExtrinsicGuess=True 时作为初始值
  - useExtrinsicGuess: 是否使用外部猜测。默认: False，如果为True，则使用提供的rvec和tvec作为初始值
  - flags: 求解方法标志，默认: `cv2.SOLVEPNP_ITERATIVE`。常用选项:
    - `cv2.SOLVEPNP_ITERATIVE` - 迭代法（默认）
    - `cv2.SOLVEPNP_EPNP` - EPnP方法
    - `cv2.SOLVEPNP_P3P` - P3P方法（需要4个点）
    - `cv2.SOLVEPNP_IPPE` - 无限平面姿态估计
    - `cv2.SOLVEPNP_IPPE_SQUARE` - 方形标记的IPPE
    - `cv2.SOLVEPNP_SQPNP` - SQPnP方法

返回值：

- retval: 成功标志。如果成功求解返回True，否则返回False
- rvec: 旋转向量。类型: np.ndarray, 形状为 (3, 1)。可以使用 `cv2.Rodrigues()` 转换为旋转矩阵
- tvec: 平移向量.类型: np.ndarray, 形状为 (3, 1).与世界坐标系单位相同（通常是米）

In [ ]:
def solve_pnp_pose(corners, camera_matrix, dist_coeffs, marker_size=0.096):
    """
    使用PnP算法求解相机相对于标记的位姿
    Args:
        corners: 标记四个角点的像素坐标
        camera_matrix: 相机内参矩阵
        dist_coeffs: 畸变系数
        marker_size: 标记边长
    Returns:
        rvec: 旋转向量
        tvec: 平移向量
    """
    # 获取世界坐标点
    world_points = get_aruco_world_coordinates(marker_size)
    # 使用PnP求解位姿
    success, rvec, tvec = cv2.solvePnP(
        world_points, 
        corners, 
        camera_matrix, 
        dist_coeffs, 
        flags=cv2.SOLVEPNP_IPPE_SQUARE
    )
    if success:
        return rvec, tvec
    else:
        return None, None

### 将求得的rvec、tvec转换为相机的世界坐标

注意，我们通过PnP求解得到的rvec和tvec是描述从世界坐标系到相机坐标系的变换。因此，要计算相机在世界坐标系中的位置，我们需要对这个变换进行逆变换

假设相机坐标系中有点$Pc$，世界坐标系中有点$Pw$，它们之间的关系为：

$$Pc = R * Pw + T$$

其中，$R$ 是由旋转向量 $rvec$ 转换得到的旋转矩阵，$T$ 是平移向量 $tvec$

那么，$Pc$ 可以通过以下公式计算：

$$Pw = R^T * (Pc - T)$$

（因为旋转矩阵是正交矩阵，其逆等于其转置，$R^T=R^{-1}$）

相机在相机坐标系中的位置为原点 $(0, 0, 0)$ ，即$Pc=(0, 0, 0)$，有：

$$Pw = -R^T * T$$

使用 `cv2.Rodrigues()` 将旋转向量转换为旋转矩阵，然后通过旋转矩阵和平移向量计算相机在世界坐标系中的位置

函数原型：

`cv2.Rodrigues(src, dst=None)`

用来将旋转向量转换为旋转矩阵，或将旋转矩阵转换为旋转向量

参数说明：

- src: 输入数组，可以是旋转向量（形状为 (3, 1) 或 (1, 3)）或旋转矩阵（形状为 (3, 3)）
- dst: 可选的输出数组，用于存储结果

返回值：

- dst: 输出数组，如果输入是旋转向量，则输出为旋转矩阵；如果输入是旋转矩阵，则输出为旋转向量
- jacobian: 可选的雅可比矩阵，用于计算导数（如果需要）

In [ ]:
rvec,tvec=solve_pnp_pose() # 仅演示
# 计算相机在标记坐标系中的位置（tvec的逆变换）
R, _ = cv2.Rodrigues(rvec)
camera_pos_in_marker = -R.T @ tvec

> 完整代码参阅`pnp_estimate.py`

## 思考

是否有必要在去畸变后的图像上进行PnP位姿估计？

没有必要

- opencv中的solvePnP()函数已经考虑了畸变校正
- 图像去畸变是一个重采样过程，会引入插值误差。特别是对于角点检测，去畸变可能会使角点位置变得模糊，降低检测精度
- 对整个图像进行去畸变需要大量计算。而PnP只需要对少数几个特征点进行畸变校正，效率更高

如果有时候的确需要对图像进行去畸变操作，那么传入solvePnP()函数的畸变参数应为全0数组，内参矩阵保持不变